In [1]:
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
import gensim.downloader as api

from Linguistic_Features import*
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from textblob import TextBlob

import pandas as pd
import pickle

unable to import 'smart_open.gcs', disabling that module


In [34]:
def Get_AvgSentence_Embedding(Sentence):
    
    sent_words = word_tokenize(Sentence)
    
    word_embeddings = [] 
    for sent_word in sent_words:
        
        if not sent_word.isalpha():
            continue
        
        if sent_word in vocabulary:
            word_embedding = vectors[sent_word]
            word_embeddings.append(word_embedding)
    
    word_embeddings = np.array(word_embeddings)
    sent_embedding = np.mean(word_embeddings, axis=0)
    
    return sent_embedding

def Load_Trained_Model(ModelPath):
    
    with open(ModelPath, "rb") as file:
        Model_Obj = pickle.load(file)
        
    return Model_Obj

def Task1_Predict(Test_Features):
    
    predictions = Task1_Model.predict(Test_Features)
    predictions_list = [int(p[0]) for p in predictions]
    
    return predictions_list

def Task2_Predict(Test_Features):
    
    ## Any preprocessing??
    predictions = Task2_Model.predict(Test_Features)
    predictions_list = [int(p[0]) for p in predictions]
    
    return predictions_list



In [56]:
def Initialization(Task1ModelPath, Task2ModelPath):
    
    global Task1_Model, Task2_Model
    
    Task1_Model = Load_Trained_Model(Task1ModelPath)
#     Task2_Model = Load_Trained_Model(Task2ModelPath)

def Bias_Detection_Suggestions(Sentences):
    
    Classification_labels = []
    
    for Sentence in Sentences:
        
        print("\n\n")
        print(Sentence)
    
        ## Step 1: Get average word embedding of the sentence
        Sent_Embedding = Get_AvgSentence_Embedding(Sentence)

        ## Step 2: Use best performing task 1 model to predict id sentence is subjective
        Sent_Embedding_Features = pd.DataFrame([Sent_Embedding])
        task1_Prediction = Task1_Predict(Sent_Embedding_Features)[0]
        
        ## Step 3: If sentence is objective: Do nothing
        if task1_Prediction==0:
            Classification_label = 0
            print("Subjectivity not identified in the sentence")
        
        ## Step 3: If sentence is subjective: Identify words inducing subjectivity in the sentence
        else:
            Classification_label = 1
            
            sent_words = word_tokenize(Sentence)
            sent_words = [w for w in sent_words if w.isalpha()]
            
            ## Step 3.1: Get liguistic features of the words in the sentence
            Sent_Linguistic_Features = Get_Sent_Linguistic_Features(Sentence)
            
            ## Ignore stopwords
            Sent_Linguistic_Features = Sent_Linguistic_Features[~Sent_Linguistic_Features['word'].isin(stop_words)]
            
            ## Ignore proper nouns (13-->NNP)
            Sent_Linguistic_Features = Sent_Linguistic_Features[Sent_Linguistic_Features['POS']!=13]
            
            ## Step 3.2: Get subjectivity score for all words in the sentence
#             Word_Subjectivity_Scores = Task2_Predict(Sent_Linguistic_Features[FeatureList])
            Word_Subjectivity_Scores = []
            for sent_word in sent_words:
                Word_Subjectivity_Scores.append(TextBlob(sent_word).sentiment.subjectivity)
    
            
            ## Step 3.3: Identify word introducing subjectivity in the sentence (max score)
            max_score = 0
            subjective_word = ""
            
            for i in range(len(sent_words)):
            
                subj_score = Word_Subjectivity_Scores[i]
                current_word = sent_words[i]
                
                if subj_score > max_score and current_word in vocabulary:
                    max_score = subj_score
                    subjective_word = current_word
                    
            if max_score == 0:
                Classification_labels.append(0)
                print("Subjectivity not identified in the sentence")
                continue
                
            print("Sentence is subjective")
            print("Word inducing subjectivity in the sentence: ", (subjective_word,max_score))
                
            ## Step 3.4: Get alternative words for identified subjective word 
            
            ## Top 10 : words with similar meaning --> words with similar embedding
            Similar_words = vectors.most_similar(subjective_word)
            Similar_words.sort(key = lambda x: x[1], reverse=True)
            
            ## Get least subjective top 3 similar words
            Subjectivity_scores = []
            
            for similar_word in Similar_words:
#                 word_features = Get_Sent_Linguistic_Features(similar_word)
#                 word_features = word_features[FeatureList]
#                 subjectivity_score = Task2_Predict(word_features)
                subjectivity_score = TextBlob(similar_word[0]).sentiment.subjectivity
                Subjectivity_scores.append((similar_word[0], subjectivity_score))
              
            Subjectivity_scores.sort(key = lambda x: x[1])
            print("Subjectivity_scores: ", Subjectivity_scores)
            
            Filter_words = [tup for tup in Subjectivity_scores if subjective_word not in tup[0] and tup[1] < max_score]
            
            Top_N = 3
            if len(Filter_words)<Top_N:
                Top_N = len(Filter_words)
            
            Top_N_Suggestions = Filter_words[:Top_N]
            
            print("Alternate word suggetions: ")
            for i in range(Top_N):
                print(Top_N_Suggestions[i][0])
         
        Classification_labels.append(Classification_label)
    return Classification_labels
        

In [4]:
## Initialization 

Folder_Path = "/Users/pranjali/Downloads/Wiki_BiasDetection"

Vocab_Filepath = Folder_Path + "/Data/Embeddings/pretrained_modified.txt"
Task1Model_Path = Folder_Path + "/Saved_Models/NN/NN_Task1_Trained_Model.pkl"
Task2Model_Path = Folder_Path + "/Saved_Models/NN/NN_Task2_Trained_Model.pkl"

vectors = KeyedVectors.load_word2vec_format(Vocab_Filepath, binary=False)
vocabulary = vectors.wv.vocab

stop_words = set(stopwords.words('english')) 

# FeatureList = ["POS", "POS_Prev", "POS_Next", "Sent_Position", "Hedge", "Hedge_Context", 
#                "Factive", "Factive_Context", "Assertive", "Assertive_Context",
#                "Implicative", "Implicative_Context", "Report", "Report_Context", 
#                 "Entailment", "Entailment_Context", "StrongSub", "StrongSub_Context", 
#                 "WeakSub", "WeakSub_Context", "Polarity", "Positive", "Positive_Context", 
#                 "Negative", "Negative_Context", "Bias_Lexicon"]

FeatureList = ["POS", "Hedge", "Factive", "Assertive", "Implicative", "Report", "Entailment", "StrongSub", 
               "WeakSub", "Polarity", "Positive", "Negative", "Bias_Lexicon"]

from Neural_Network import*
Initialization(Task1Model_Path, Task2Model_Path)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.
Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/l

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [46]:
## Testing

Test_Filepath = Folder_Path + "/Data/Model_Data/test_sentences.csv"
Test_Data = pd.read_csv(Test_Filepath)

Test_Filtered_Data = []
for i in range(Test_Data.shape[0]):
    row = Test_Data.iloc[i]
    
    if len(row['sentence'])>10:
        Test_Filtered_Data.append(row)

Test_Filtered_Data = pd.DataFrame(Test_Filtered_Data, columns=['sentence', 'label'])

Test_Sentences = Test_Filtered_Data['sentence']
Test_Labels = Test_Filtered_Data['label']


# Test_Sentences = [
#                   "nicaea was convoked by the emperor constantine in mayaugust 325 to address the arian heresy that christ is of a distinct substance from the father.",
#                   "dennis the menace is an american animated series produced by dic entertainment, based on the classic comic strip by hank ketcham.",
#                   "speculative strategies of life extension.",
#                   "later editions of the work were famously illustrated by the norwegian artists erik werenskiold, theodor kittelsen, otto sinding and others.",
#                   "blogger supports google's adsense service as a simple way of generating revenue from running a blog.",
#                   "she was married to emil beaulieau, america's greatest living noisician.",
#                   "what's worse is that pelosi knows it.",
#                   "For a few frequently occurring network motifs we show how they manage noise."
#                  ]

Classification_labels = Bias_Detection_Suggestions(Test_Sentences)





what's worse is that pelosi knows it .
1/1 [==============================] - 0s 501us/step
Subjectivity not identified in the sentence



the filmmakers wisely decided to let crocodile hunter steve irwin do what he does best , and fashion a story around him .
1/1 [==============================] - 0s 359us/step
Subjectivity not identified in the sentence



For a few frequently occurring network motifs we show how they manage noise.
1/1 [==============================] - 0s 604us/step
Subjectivity not identified in the sentence



and in the dogs , she finds lost pieces of herself as she fumbles with the leash to her own life - struggling to hold on as it pulls her toward a brighter future before her past catches up with her .
1/1 [==============================] - 0s 841us/step
Subjectivity not identified in the sentence



Suggests two approaches to economics of blockchain: innovation-centred and governance-centred.
1/1 [==============================] - 0s 1ms/step
Subjectivity 

Subjectivity_scores:  [('one', 0.0), ('it', 0.0), ('but', 0.0), ('though', 0.0), ('not', 0.0), ('this', 0.0), ('there', 0.0), ('even', 0.0), ('all', 0.0), ('same', 0.125)]
Alternate word suggetions: 
one
it
but



against improbable odds , they race towards a thrilling , climactic confrontation on the mysterious isla de muerta .
1/1 [==============================] - 0s 939us/step
Subjectivity not identified in the sentence



it's a great performance and a reminder of dickens' grandeur .
1/1 [==============================] - 0s 445us/step
Sentence is subjective
[('it', 'PRP'), ('a', 'DT'), ('great', 'JJ'), ('performance', 'NN'), ('and', 'CC'), ('a', 'DT'), ('reminder', 'NN'), ('of', 'IN'), ('dickens', 'NNS'), ('grandeur', 'NN')]
Word inducing subjectivity in the sentence:  great
Subjectivity_scores:  [('well', 0.0), ('there', 0.0), ('way', 0.0), ('big', 0.1), ('much', 0.2), ('little', 0.5), ('good', 0.6000000000000001), ('kind', 0.9), ('greatest', 1.0), ('important', 1.0)]
Alternate 

1/1 [==============================] - 0s 544us/step
Sentence is subjective
[('a', 'DT'), ('skillfully', 'RB'), ('made', 'VBN'), ('example', 'NN'), ('of', 'IN'), ('your', 'PRP$'), ('typical', 'JJ'), ('schwarzenegger', 'NN'), ('action', 'NN'), ('film', 'NN')]
Word inducing subjectivity in the sentence:  typical
Subjectivity_scores:  [('example', 0.0), ('instance', 0.0), ('contrast', 0.0), ('depending', 0.0), ('unlike', 0.0), ('usual', 0.25), ('usually', 0.25), ('simple', 0.35714285714285715), ('typically', 0.5), ('unusual', 1.0)]
Alternate word suggetions: 
example
instance
contrast



based on a devilishly witty script by heather mcgowan and niels mueller , the film gets great laughs , but never at the expense of its characters
1/1 [==============================] - 0s 432us/step
Sentence is subjective
[('based', 'VBN'), ('on', 'IN'), ('a', 'DT'), ('devilishly', 'JJ'), ('witty', 'JJ'), ('script', 'NN'), ('by', 'IN'), ('heather', 'NN'), ('mcgowan', 'NN'), ('and', 'CC'), ('niels', 'NNS')

[('credit', 'NN'), ('director', 'NN'), ('ramsay', 'VBP'), ('for', 'IN'), ('taking', 'VBG'), ('the', 'DT'), ('sometimes', 'RB'), ('improbable', 'JJ'), ('story', 'NN'), ('and', 'CC'), ('making', 'VBG'), ('it', 'PRP'), ('feel', 'VB'), ('realistic', 'JJ')]
Word inducing subjectivity in the sentence:  realistic
Subjectivity_scores:  [('practical', 0.0), ('sensible', 0.0), ('nuanced', 0.0), ('objective', 0.1), ('straightforward', 0.375), ('plausible', 0.5), ('reasonable', 0.6), ('compelling', 0.6), ('accurate', 0.6333333333333334), ('unrealistic', 1.0)]
Alternate word suggetions: 
practical
sensible
nuanced



ah , what the hell .
1/1 [==============================] - 0s 558us/step
Subjectivity not identified in the sentence



, is discontented with almost everything in life despite his popularity and the love of his girlfriend , grace ( aniston ) .
1/1 [==============================] - 0s 400us/step
Subjectivity not identified in the sentence



but things go very wrong when he tries thi

1/1 [==============================] - 0s 2ms/step
Subjectivity not identified in the sentence



this is scorsese's finest work , and one of the best films of 2002 .
1/1 [==============================] - 0s 628us/step
Sentence is subjective
[('this', 'DT'), ('is', 'VBZ'), ('scorsese', 'JJ'), ('finest', 'JJS'), ('work', 'NN'), ('and', 'CC'), ('one', 'CD'), ('of', 'IN'), ('the', 'DT'), ('best', 'JJS'), ('films', 'NNS'), ('of', 'IN')]
Word inducing subjectivity in the sentence:  best
Subjectivity_scores:  [('well', 0.0), ('one', 0.0), ('performance', 0.0), ('success', 0.0), ('ever', 0.0), ('play', 0.0), ('better', 0.5), ('good', 0.6000000000000001), ('winning', 0.75), ('excellent', 1.0)]
Alternate word suggetions: 
well
one
performance



if the film fails to fulfill its own ambitious goals , it nonetheless sustains interest during the long build-up of expository material .
1/1 [==============================] - 0s 542us/step
Sentence is subjective
[('if', 'IN'), ('the', 'DT'), ('film',

...

Subjectivity not identified in the sentence



this isn't a terrible film by any means , but it's also far from being a realized work .
1/1 [==============================] - 0s 791us/step
Subjectivity not identified in the sentence



it's packed with adventure and a worthwhile environmental message , so it's great for the kids .
1/1 [==============================] - 0s 463us/step
Sentence is subjective
[('it', 'PRP'), ('packed', 'VBN'), ('with', 'IN'), ('adventure', 'NN'), ('and', 'CC'), ('a', 'DT'), ('worthwhile', 'JJ'), ('environmental', 'JJ'), ('message', 'NN'), ('so', 'IN'), ('it', 'PRP'), ('great', 'JJ'), ('for', 'IN'), ('the', 'DT'), ('kids', 'NNS')]
Word inducing subjectivity in the sentence:  great
Subjectivity_scores:  [('well', 0.0), ('there', 0.0), ('way', 0.0), ('big', 0.1), ('much', 0.2), ('little', 0.5), ('good', 0.6000000000000001), ('kind', 0.9), ('greatest', 1.0), ('important', 1.0)]
Alternate word suggetions: 
well
there
way



The development of the atmospheric Ch

Subjectivity not identified in the sentence



viveka is a glamorous woman and has high standards of living .
1/1 [==============================] - 0s 2ms/step
Subjectivity not identified in the sentence



we never truly come to care about the main characters and whether or not they'll wind up together , and michele's spiritual quest is neither amusing nor dramatic enough to sustain interest .
1/1 [==============================] - 0s 887us/step
Subjectivity not identified in the sentence



but when the teenager decides to invite a 12-year-old boy who is running away from home , maud is near illegality .
1/1 [==============================] - 0s 451us/step
Subjectivity not identified in the sentence



We present algorithms for the group independent reduction of group theory factors of Feynman diagrams.
1/1 [==============================] - 0s 2ms/step
Subjectivity not identified in the sentence



sam mendes has become valedictorian at the school for soft landings and easy ways ou

Subjectivity_scores:  [('pleasing', 0.0), ('gratifying', 0.0), ('straightforward', 0.375), ('interesting', 0.5), ('enjoyable', 0.6), ('compelling', 0.6), ('entertaining', 0.7), ('exciting', 0.8), ('exhilarating', 0.9), ('rewarding', 1.0)]
Alternate word suggetions: 
pleasing
gratifying
straightforward



Drawing from available examples of blockchain uses in education, the paper puts forward the concept of 'knowledge procurement' to develop a decentralised learning ledger (DLL) for the tracking, certifying and practical use of learning credits.
1/1 [==============================] - 0s 413us/step
Subjectivity not identified in the sentence



plays like a checklist of everything rob reiner and his cast were sending up .
1/1 [==============================] - 0s 512us/step
Subjectivity not identified in the sentence



an exciting debut from promising , and fairly disturbed , young director/screenwriter dylan kidd .
1/1 [==============================] - 0s 389us/step
Sentence is subject

Subjectivity not identified in the sentence



his songs are no longer seen as top grade .
1/1 [==============================] - 0s 1ms/step
Subjectivity not identified in the sentence



Suicide is the second leading cause of death in the 15-29 years age group, and its prevention is one of the top public health priorities.
1/1 [==============================] - 0s 477us/step
Subjectivity not identified in the sentence



excellent acting and direction .
1/1 [==============================] - 0s 347us/step
Sentence is subjective
[('excellent', 'JJ'), ('acting', 'NN'), ('and', 'CC'), ('direction', 'NN')]
Word inducing subjectivity in the sentence:  excellent
Subjectivity_scores:  [('quality', 0.0), ('solid', 0.1), ('best', 0.3), ('better', 0.5), ('good', 0.6000000000000001), ('decent', 0.6666666666666666), ('terrific', 1.0), ('superb', 1.0), ('impressive', 1.0), ('perfect', 1.0)]
Alternate word suggetions: 
quality
solid
best



you have no affinity for most of the characters .
1/1 [==

Subjectivity_scores:  [('one', 0.0), ('it', 0.0), ('but', 0.0), ('though', 0.0), ('not', 0.0), ('this', 0.0), ('there', 0.0), ('even', 0.0), ('all', 0.0), ('same', 0.125)]
Alternate word suggetions: 
one
it
but



In this paper, we propose a new, fermionic Group Field Theory, posessing a color symmetry, and take the first steps in a systematic study of the topological properties of its graphs.
1/1 [==============================] - 0s 528us/step
Subjectivity not identified in the sentence



rooney in the past is sent to kill sullivan .
1/1 [==============================] - 0s 517us/step
Subjectivity not identified in the sentence



shot on location in the williamsburgh and flatbush neighborhoods of brooklyn , hung-up is a 35mm short story in the tradition of happy gilmore and taxi driver .
1/1 [==============================] - 0s 396us/step
Subjectivity not identified in the sentence



she entranced the founder of the museum of art ( masp ) where the fashion parade was being held 

In [48]:
print("Test_Filtered_Data: ", Test_Filtered_Data.shape)

Platinum_Dataset = []

for i in range(Test_Filtered_Data.shape[0]):
    Actual_label = Test_Labels.iloc[i]
    Predicted_label = Classification_labels[i]
    
    if Actual_label == Predicted_label:
        sent = Test_Filtered_Data.iloc[i]['sentence']
        Platinum_Dataset.append(sent)
        
Platinum_Dataset = pd.DataFrame(Platinum_Dataset, columns=['sentence'])
Platinum_Dataset.to_csv(Folder_Path + "/Data/Model_Data/FinalTestData.csv")

print("Platinum_Dataset: ", Platinum_Dataset.shape)

Test_Filtered_Data:  (3072, 2)
Platinum_Dataset:  (2244, 1)


In [57]:
## Testing

Test_Filepath = Folder_Path + "/Data/Model_Data/FinalTestData.csv"
Test_Data = pd.read_csv(Test_Filepath)
Test_Sentences = Test_Data['sentence']

Classification_labels = Bias_Detection_Suggestions(Test_Sentences)





For a few frequently occurring network motifs we show how they manage noise.
1/1 [==============================] - 0s 1ms/step
Subjectivity not identified in the sentence



and in the dogs , she finds lost pieces of herself as she fumbles with the leash to her own life - struggling to hold on as it pulls her toward a brighter future before her past catches up with her .
1/1 [==============================] - 0s 930us/step
Subjectivity not identified in the sentence



Suggests two approaches to economics of blockchain: innovation-centred and governance-centred.
1/1 [==============================] - 0s 499us/step
Subjectivity not identified in the sentence



being blue is not his only predicament ; he also lacks the ability to outwardly express his emotions .
1/1 [==============================] - 0s 570us/step
Subjectivity not identified in the sentence



on her deathbed , candice klein accidentally asks the question , " what did i ever do to deserve this ? "
1/1 [=============

[('making', 'VBG'), ('such', 'JJ'), ('a', 'DT'), ('tragedy', 'NN'), ('the', 'DT'), ('backdrop', 'NN'), ('to', 'TO'), ('a', 'DT'), ('love', 'JJ'), ('story', 'NN'), ('risks', 'NNS'), ('trivializing', 'VBG'), ('it', 'PRP'), ('though', 'IN'), ('chouraqui', 'JJ'), ('no', 'DT'), ('doubt', 'NN'), ('intended', 'VBD'), ('the', 'DT'), ('film', 'NN'), ('to', 'TO'), ('affirm', 'VB'), ('love', 'NN'), ('power', 'NN'), ('to', 'TO'), ('help', 'VB'), ('people', 'NNS'), ('endure', 'VB'), ('almost', 'RB'), ('unimaginable', 'JJ'), ('horror', 'NN')]
Sentence is subjective
Word inducing subjectivity in the sentence:  ('love', 0.6)
Subjectivity_scores:  [('me', 0.0), ('passion', 0.0), ('my', 0.0), ('life', 0.0), ('dream', 0.0), ('you', 0.0), ('always', 0.0), ('wonder', 0.0), ('i', 0.0), ('dreams', 0.0)]
Alternate word suggetions: 
me
passion
my



Last month Digital Science also published a report about the potential use of blockchain technology for scholarly communication and, as detailed in Information Tod

Subjectivity not identified in the sentence



when she fails to find the " perfect man " that will make her happy , she decides to construct him herself , using parts from different people .
1/1 [==============================] - 0s 379us/step
Subjectivity not identified in the sentence



It is shown that one version of the problem is intractable, while another is solvable by an efficient algorithm.
1/1 [==============================] - 0s 457us/step
Subjectivity not identified in the sentence



but harper himself is a suspect in several bomb attacks , one of which killed rachel's partner and friend .
1/1 [==============================] - 0s 407us/step
Subjectivity not identified in the sentence



the driver finds himself in a dangerous , yet highly political situation ; this time being pursued by a helicopter gunship while carrying a passenger with a suitcase .
1/1 [==============================] - 0s 440us/step
Subjectivity not identified in the sentence



 " the ballad of be

1/1 [==============================] - 0s 1ms/step
Subjectivity not identified in the sentence



enlisted for a dangerous covert mission , he must combat a clever , organized , and ruthless enemy far beyond the scope of his experience .
1/1 [==============================] - 0s 546us/step
Subjectivity not identified in the sentence



to complete the deal , he must bring a young american hustler boy ( who is taken to rio not knowing that he'll be killed in a snuff film ) .
1/1 [==============================] - 0s 601us/step
Subjectivity not identified in the sentence



It is shown that in general the integrated renormalisation group flows fail to reproduce perturbation theory beyond one loop.
1/1 [==============================] - 0s 635us/step
Subjectivity not identified in the sentence



he has been detained there as a result of planted incriminating evidence of unlawful investigation techniques .
1/1 [==============================] - 0s 613us/step
Subjectivity not identified in

1/1 [==============================] - 0s 3ms/step
Subjectivity not identified in the sentence



Sagesse  had previously won eight straight games at the beginning of the season, and Champville looked on course to tie that record from the outset.
1/1 [==============================] - 0s 504us/step
Subjectivity not identified in the sentence



Several examples are presented where classical topology is changed by smoothly altering the boundary conditions.
1/1 [==============================] - 0s 915us/step
Subjectivity not identified in the sentence



a man brutally beats his girlfriend to death and tosses her remains into a local river .
1/1 [==============================] - 0s 5ms/step
Subjectivity not identified in the sentence



though perry and hurley make inspiring efforts to breathe life into the disjointed , haphazard script by jay scherick and david ronn , neither the actors nor director reginald hudlin can make it more than fitfully entertaining .
1/1 [===================

...

Subjectivity not identified in the sentence



although it's a bit smug and repetitive , this documentary engages your brain in a way few current films do .
1/1 [==============================] - 0s 528us/step
[('although', 'IN'), ('it', 'PRP'), ('a', 'DT'), ('bit', 'NN'), ('smug', 'JJ'), ('and', 'CC'), ('repetitive', 'JJ'), ('this', 'DT'), ('documentary', 'JJ'), ('engages', 'VBZ'), ('your', 'PRP$'), ('brain', 'NN'), ('in', 'IN'), ('a', 'DT'), ('way', 'NN'), ('few', 'JJ'), ('current', 'JJ'), ('films', 'NNS'), ('do', 'VBP')]
Sentence is subjective
Word inducing subjectivity in the sentence:  ('current', 0.4)
Subjectivity_scores:  [('the', 0.0), ('present', 0.0), ('change', 0.0), ('this', 0.0), ('its', 0.0), ('term', 0.0), ('same', 0.125), ('future', 0.125), ('previous', 0.16666666666666666), ('new', 0.45454545454545453)]
Alternate word suggetions: 
the
present
change



the deuces are determined to do whatever is necessary to keep drugs off their block even if that means dying .
1/1 [==

Sentence is subjective
Word inducing subjectivity in the sentence:  ('satisfying', 1.0)
Subjectivity_scores:  [('pleasing', 0.0), ('gratifying', 0.0), ('straightforward', 0.375), ('interesting', 0.5), ('enjoyable', 0.6), ('compelling', 0.6), ('entertaining', 0.7), ('exciting', 0.8), ('exhilarating', 0.9), ('rewarding', 1.0)]
Alternate word suggetions: 
pleasing
gratifying
straightforward



and what makes lucky invaluable is his ability to teach mudd how to write again .
1/1 [==============================] - 0s 748us/step
Subjectivity not identified in the sentence



straight up : helicopters in action will take audiences on a series of aerial adventures .
1/1 [==============================] - 0s 1ms/step
Subjectivity not identified in the sentence



Those 3-pointers put him in the record book.
1/1 [==============================] - 0s 431us/step
Subjectivity not identified in the sentence



the first shocking thing about sorority boys is that it's actually watchable .
1/1 [======

1/1 [==============================] - 0s 750us/step
Subjectivity not identified in the sentence



the government sends in an elite military task force lead by alice ( jovovich ) and rain ( rodriguez ) to contain the virus in three hours before it escapes and infects the rest of the world .
1/1 [==============================] - 0s 2ms/step
Subjectivity not identified in the sentence



viveka is a glamorous woman and has high standards of living .
1/1 [==============================] - 0s 1ms/step
Subjectivity not identified in the sentence



but when the teenager decides to invite a 12-year-old boy who is running away from home , maud is near illegality .
1/1 [==============================] - 0s 1ms/step
Subjectivity not identified in the sentence



We present algorithms for the group independent reduction of group theory factors of Feynman diagrams.
1/1 [==============================] - 0s 490us/step
Subjectivity not identified in the sentence



an excellent sequel .
1/1 [====

1/1 [==============================] - 0s 404us/step
[('an', 'DT'), ('exciting', 'JJ'), ('debut', 'NN'), ('from', 'IN'), ('promising', 'NN'), ('and', 'CC'), ('fairly', 'RB'), ('disturbed', 'VBN'), ('young', 'JJ'), ('dylan', 'NN'), ('kidd', 'NN')]
Sentence is subjective
Word inducing subjectivity in the sentence:  ('fairly', 0.9)
Subjectivity_scores:  [('quite', 0.0), ('relatively', 0.0), ('equally', 0.25), ('very', 0.3), ('similarly', 0.4), ('reasonably', 0.6), ('remarkably', 0.75), ('extremely', 1.0), ('pretty', 1.0), ('exceptionally', 1.0)]
Alternate word suggetions: 
quite
relatively
equally



Simplification is induced by confluent rewriting on terms.
1/1 [==============================] - 0s 454us/step
Subjectivity not identified in the sentence



Quantum physics and biology have long been regarded as unrelated disciplines, describing nature at the inanimate microlevel on the one hand and living species on the other hand.
1/1 [==============================] - 0s 498us/step
Subje

Subjectivity not identified in the sentence



it's still a comic book , but maguire makes it a comic book with soul .
1/1 [==============================] - 0s 992us/step
[('it', 'PRP'), ('still', 'RB'), ('a', 'DT'), ('comic', 'JJ'), ('book', 'NN'), ('but', 'CC'), ('maguire', 'NN'), ('makes', 'VBZ'), ('it', 'PRP'), ('a', 'DT'), ('comic', 'JJ'), ('book', 'NN'), ('with', 'IN'), ('soul', 'NN')]
Sentence is subjective
Word inducing subjectivity in the sentence:  ('comic', 0.5)
Subjectivity_scores:  [('comics', 0.0), ('fiction', 0.0), ('superhero', 0.0), ('comedy', 0.0), ('character', 0.0), ('parody', 0.0), ('characters', 0.0), ('comedic', 0.0), ('animated', 0.0), ('cartoon', 0.0)]
Alternate word suggetions: 
fiction
superhero
comedy



stiles has an ulterior motive however , and has enlisted possum lake's sheriff ( darren frost ) and deputy ( melissa dimarco ) to help prevent the lodge from coming up with money to pay the fine .
1/1 [==============================] - 0s 599us/step
Subject

In [51]:
Classification_labels0 = [l for l in Classification_labels if l==0]
print("Objective: ", len(Classification_labels0))
print("Subjective: ", len(Classification_labels)-len(Classification_labels0))

Objective:  1666
Subjective:  578
